# Calculate potential Evapo-transpiration (PET) girds
The purpose of this notebook is to create PET grids as part of the Cambodia/eWater project. The notebook uses daily min,mean and max air temperature grids and the Hargreaves equation.

## Import Libraries

In [6]:
import numpy as np
import math as mt
from datetime import *
import calendar
import netCDF4 as nc
import xarray as xr
import glob

## Create Functions

In [3]:
#Create function to calculate date from netcdf (as units of netcdf are "seconds since 20xx-xx-xx 00:00:00" )
def to_date(epoch, time_unit):
    time_unit =  datetime.strptime(time_unit, '%Y-%m-%d %H:%M:%S') 
    return time_unit + timedelta(seconds=epoch)

#Create function to calulate julian day of year (days since start of year)
def julian_days(epoch, time_unit):
    start_of_year = datetime.strptime('01/01/'+ str(time_unit),'%m/%d/%Y')
    start_of_month = datetime.strptime(min_xr.time.units[14:], '%Y-%m-%d %H:%M:%S') 
    epoch = start_of_month + timedelta(seconds=epoch)
    j_days = int((((epoch - start_of_year)+timedelta(days=1))/timedelta(days=1)))
    return (j_days)

#Create function to test whether the year of interest is a leap year
def leap_year(year):
    if calendar.isleap(int(year)) == True:
#         print(str(year) +' is a leap year')
        ndy = 366
    else:
#         print(str(year) +' is not a leap year')
        ndy = 365
    return(ndy)
    

## loop through all temperature grids and save as netCDF format

In [4]:
input_folder = '/g/data/u46/users/ext547/ewater/input_data/Temperature/degrees_celcius_and_clipped/celcius/' #location of files
ncfiles = glob.glob(input_folder + 'original/*.nc') #a glob file containing the names of all nc

for ncpath in ncfiles: #loop through all nc
    #Set dataset variables
    nc_name = ncpath[-28:]
    print ('Processing '+str(nc_name)+' now')
    min_nc = 'min_'+str(nc_name)
    max_nc = 'max_'+str(nc_name)
    mean_nc = 'mean_'+str(nc_name)
    
    #Input temperature datasets
    min_xr = xr.open_dataset(input_folder+'/min/'+min_nc, decode_times=False)
    mean_xr = xr.open_dataset(input_folder+'/mean/'+mean_nc, decode_times=False)
    max_xr = xr.open_dataset(input_folder+'/max/'+max_nc, decode_times=False)
    
    #Calculate date from netcdf and save as xarray dataset
    time = xr.DataArray([to_date(epoch, min_xr.time.units[14:]) for epoch in min_xr.time.values.tolist()], coords={'time': min_xr.time}, dims=('time',))
   
    #Calculate julian days and save as an array
    j_days = xr.DataArray([julian_days(epoch, 
                                       min_xr.time.units[14:18]) for epoch in time.time.values.tolist()], 
                          coords={'time': min_xr.time}, dims=('time',))
    
    #Use leap year function and save the number of days within the year as a variable
    ndy = leap_year( min_xr.time.units[14:18])
    
    #Create array of temperature difference
    #Unit: T_diff in degrees celcius
    T_diff = max_xr.Tair - min_xr.Tair
    
    #Create array of latitude in radians
    #Unit: lat_rad in radians
    lat_rad = np.radians(min_xr.lat)
    
    # Caclulate the inverse relative distance between the Earth and the Sun
    # Unit: dr in radians
    dr = 1 + 0.033 * np.cos(2 * np.pi * j_days / ndy)
    
    # Calculate solar declination
    # Unit: sd in radians
    sd = 0.409 * np.sin(2 * mt.pi * j_days / ndy - 1.39)

    # Calculate a sunset angle input variable
    # Unit: no unit
    sa_X = 1 - (np.tan(lat_rad))**2 * ((np.tan(sd))**2)
    
    # Calculate sunset angle
    # Units: sa in radians
    sa = mt.pi / 2 - np.arctan(-np.tan(lat_rad) * np.tan(sd) / np.sqrt(sa_X))

    # Calcualte extraterrestrial radiation
    # Units: ra in MJ / (m^ day)
    ra = 37.586 * dr * (sa * np.sin(lat_rad) * np.sin(sd) + np.cos(lat_rad) * np.cos(sd) * np.sin(sa))
    
    # Calculate Potential Evapotranspiration
    # Units: PET in mm/day
    pet = 0.0009384 * (mean_xr.Tair + 17.8) * np.sqrt(T_diff) * ra
    pet = xr.Dataset({'PET':pet})
    pet.PET.attrs['units'] = 'mm/day' #add unit to xarray
    
    #Save PET output as netcdf
    pet.to_netcdf(path = '/g/data/u46/users/ext547/ewater/input_data/PET/PET'+nc_name[12:])
    print ('PET'+str(nc_name[12:])+' is complete')

Processing celsius_Tair_WFDEI_198902.nc now
PET_WFDEI_198902.nc is complete
Processing celsius_Tair_WFDEI_199102.nc now
PET_WFDEI_199102.nc is complete
Processing celsius_Tair_WFDEI_198310.nc now
PET_WFDEI_198310.nc is complete
Processing celsius_Tair_WFDEI_200904.nc now
PET_WFDEI_200904.nc is complete
Processing celsius_Tair_WFDEI_199609.nc now
PET_WFDEI_199609.nc is complete
Processing celsius_Tair_WFDEI_201608.nc now
PET_WFDEI_201608.nc is complete
Processing celsius_Tair_WFDEI_199207.nc now
PET_WFDEI_199207.nc is complete
Processing celsius_Tair_WFDEI_198012.nc now
PET_WFDEI_198012.nc is complete
Processing celsius_Tair_WFDEI_201205.nc now
PET_WFDEI_201205.nc is complete
Processing celsius_Tair_WFDEI_198408.nc now
PET_WFDEI_198408.nc is complete
Processing celsius_Tair_WFDEI_199803.nc now
PET_WFDEI_199803.nc is complete
Processing celsius_Tair_WFDEI_200212.nc now
PET_WFDEI_200212.nc is complete
Processing celsius_Tair_WFDEI_201009.nc now
PET_WFDEI_201009.nc is complete
Processing c

PET_WFDEI_199812.nc is complete
Processing celsius_Tair_WFDEI_198008.nc now
PET_WFDEI_198008.nc is complete
Processing celsius_Tair_WFDEI_200102.nc now
PET_WFDEI_200102.nc is complete
Processing celsius_Tair_WFDEI_200002.nc now
PET_WFDEI_200002.nc is complete
Processing celsius_Tair_WFDEI_201607.nc now
PET_WFDEI_201607.nc is complete
Processing celsius_Tair_WFDEI_200109.nc now
PET_WFDEI_200109.nc is complete
Processing celsius_Tair_WFDEI_199701.nc now
PET_WFDEI_199701.nc is complete
Processing celsius_Tair_WFDEI_198909.nc now
PET_WFDEI_198909.nc is complete
Processing celsius_Tair_WFDEI_200410.nc now
PET_WFDEI_200410.nc is complete
Processing celsius_Tair_WFDEI_199212.nc now
PET_WFDEI_199212.nc is complete
Processing celsius_Tair_WFDEI_199505.nc now
PET_WFDEI_199505.nc is complete
Processing celsius_Tair_WFDEI_198903.nc now
PET_WFDEI_198903.nc is complete
Processing celsius_Tair_WFDEI_200110.nc now
PET_WFDEI_200110.nc is complete
Processing celsius_Tair_WFDEI_198505.nc now
PET_WFDEI_19

PET_WFDEI_198611.nc is complete
Processing celsius_Tair_WFDEI_199903.nc now
PET_WFDEI_199903.nc is complete
Processing celsius_Tair_WFDEI_199406.nc now
PET_WFDEI_199406.nc is complete
Processing celsius_Tair_WFDEI_198806.nc now
PET_WFDEI_198806.nc is complete
Processing celsius_Tair_WFDEI_199204.nc now
PET_WFDEI_199204.nc is complete
Processing celsius_Tair_WFDEI_200511.nc now
PET_WFDEI_200511.nc is complete
Processing celsius_Tair_WFDEI_198801.nc now
PET_WFDEI_198801.nc is complete
Processing celsius_Tair_WFDEI_199209.nc now
PET_WFDEI_199209.nc is complete
Processing celsius_Tair_WFDEI_201208.nc now
PET_WFDEI_201208.nc is complete
Processing celsius_Tair_WFDEI_199004.nc now
PET_WFDEI_199004.nc is complete
Processing celsius_Tair_WFDEI_198606.nc now
PET_WFDEI_198606.nc is complete
Processing celsius_Tair_WFDEI_199605.nc now
PET_WFDEI_199605.nc is complete
Processing celsius_Tair_WFDEI_200107.nc now
PET_WFDEI_200107.nc is complete
Processing celsius_Tair_WFDEI_200801.nc now
PET_WFDEI_20

PET_WFDEI_199601.nc is complete
Processing celsius_Tair_WFDEI_198502.nc now
PET_WFDEI_198502.nc is complete
Processing celsius_Tair_WFDEI_198209.nc now
PET_WFDEI_198209.nc is complete
Processing celsius_Tair_WFDEI_198411.nc now
PET_WFDEI_198411.nc is complete
Processing celsius_Tair_WFDEI_200508.nc now
PET_WFDEI_200508.nc is complete
Processing celsius_Tair_WFDEI_199906.nc now
PET_WFDEI_199906.nc is complete
Processing celsius_Tair_WFDEI_201004.nc now
PET_WFDEI_201004.nc is complete
Processing celsius_Tair_WFDEI_199002.nc now
PET_WFDEI_199002.nc is complete
Processing celsius_Tair_WFDEI_199508.nc now
PET_WFDEI_199508.nc is complete
Processing celsius_Tair_WFDEI_198403.nc now
PET_WFDEI_198403.nc is complete
Processing celsius_Tair_WFDEI_200009.nc now
PET_WFDEI_200009.nc is complete
Processing celsius_Tair_WFDEI_200011.nc now
PET_WFDEI_200011.nc is complete
Processing celsius_Tair_WFDEI_200906.nc now
PET_WFDEI_200906.nc is complete
Processing celsius_Tair_WFDEI_199111.nc now
PET_WFDEI_19

PET_WFDEI_199802.nc is complete
Processing celsius_Tair_WFDEI_198804.nc now
PET_WFDEI_198804.nc is complete
Processing celsius_Tair_WFDEI_201204.nc now
PET_WFDEI_201204.nc is complete
Processing celsius_Tair_WFDEI_198512.nc now
PET_WFDEI_198512.nc is complete
Processing celsius_Tair_WFDEI_200812.nc now
PET_WFDEI_200812.nc is complete
Processing celsius_Tair_WFDEI_198404.nc now
PET_WFDEI_198404.nc is complete
Processing celsius_Tair_WFDEI_199008.nc now
PET_WFDEI_199008.nc is complete
Processing celsius_Tair_WFDEI_201504.nc now
PET_WFDEI_201504.nc is complete
Processing celsius_Tair_WFDEI_198305.nc now
PET_WFDEI_198305.nc is complete
Processing celsius_Tair_WFDEI_200612.nc now
PET_WFDEI_200612.nc is complete
Processing celsius_Tair_WFDEI_198506.nc now
PET_WFDEI_198506.nc is complete
Processing celsius_Tair_WFDEI_198907.nc now
PET_WFDEI_198907.nc is complete
Processing celsius_Tair_WFDEI_198906.nc now
PET_WFDEI_198906.nc is complete
Processing celsius_Tair_WFDEI_199606.nc now
PET_WFDEI_19

In [230]:
#Set dataset variables
input_folder = '/g/data/u46/users/ext547/ewater/input_data/test/'
nc_name = 'celsius_Tair_WFDEI_197901.nc'
min_nc = 'min_'+str(nc_name)
max_nc = 'max_'+str(nc_name)
mean_nc = 'mean_'+str(nc_name)

In [231]:
#Input temperature datasets
min_xr = xr.open_dataset(input_folder+min_nc, decode_times=False)
mean_xr = xr.open_dataset(input_folder+mean_nc, decode_times=False)
max_xr = xr.open_dataset(input_folder+max_nc, decode_times=False)

In [232]:
#Check array
print(mean_xr)

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 13, lon: 14, time: 31)
Coordinates:
  * time       (time) float64 3.78e+04 1.242e+05 2.106e+05 2.97e+05 ...
  * lon        (lon) float32 101.75 102.25 102.75 103.25 103.75 104.25 ...
  * lat        (lat) float32 9.25 9.75 10.25 10.75 11.25 11.75 12.25 12.75 ...
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) float64 ...
    Tair       (time, lat, lon) float64 ...
    timestp    (time) int32 ...
Attributes:
    CDI:                       Climate Data Interface version 1.9.2 (http://m...
    Conventions:               CF-1.6
    history:                   Tue Jun 12 16:04:56 2018: cdo daymean /g/data/...
    Title:                     WATCH Forcing Data methodology applied \nto ER...
    Note1:                     EI data regridded to half degree regular lat-lon
    Note2:                     Genuine land points from CRU grid only
    Note3:                     Tair elevation & bias-corrected using \nCRU TS..

# Reformat date variable

In [233]:
#Create function to calculate date from netcdf (as units of netcdf are "seconds since 20xx-xx-xx 00:00:00" )
def to_date(epoch, time_unit):
    time_unit =  datetime.strptime(time_unit, '%Y-%m-%d %H:%M:%S') 
    return time_unit + timedelta(seconds=epoch)
#Calculate date from netcdf and save as xarray dataset
time = xr.DataArray([to_date(epoch, min_xr.time.units[14:]) for epoch in min_xr.time.values.tolist()], coords={'time': min_xr.time}, dims=('time',))
print(time)

<xarray.DataArray (time: 31)>
array(['1979-01-01T10:30:00.000000000', '1979-01-02T10:30:00.000000000',
       '1979-01-03T10:30:00.000000000', '1979-01-04T10:30:00.000000000',
       '1979-01-05T10:30:00.000000000', '1979-01-06T10:30:00.000000000',
       '1979-01-07T10:30:00.000000000', '1979-01-08T10:30:00.000000000',
       '1979-01-09T10:30:00.000000000', '1979-01-10T10:30:00.000000000',
       '1979-01-11T10:30:00.000000000', '1979-01-12T10:30:00.000000000',
       '1979-01-13T10:30:00.000000000', '1979-01-14T10:30:00.000000000',
       '1979-01-15T10:30:00.000000000', '1979-01-16T10:30:00.000000000',
       '1979-01-17T10:30:00.000000000', '1979-01-18T10:30:00.000000000',
       '1979-01-19T10:30:00.000000000', '1979-01-20T10:30:00.000000000',
       '1979-01-21T10:30:00.000000000', '1979-01-22T10:30:00.000000000',
       '1979-01-23T10:30:00.000000000', '1979-01-24T10:30:00.000000000',
       '1979-01-25T10:30:00.000000000', '1979-01-26T10:30:00.000000000',
       '1979-01-27T10

In [234]:
#Create function to calulate julian day of year (days since start of year)
def julian_days(epoch, time_unit):
    start_of_year = datetime.strptime('01/01/'+ str(time_unit),'%m/%d/%Y')
    start_of_month = datetime.strptime(min_xr.time.units[14:], '%Y-%m-%d %H:%M:%S') 
    epoch = start_of_month + timedelta(seconds=epoch)
    j_days = int((((epoch - start_of_year)+timedelta(days=1))/timedelta(days=1)))
    return (j_days)
#Calculate julian days and save as an array
j_days = xr.DataArray([julian_days(epoch, 
                                   min_xr.time.units[14:18]) for epoch in time.time.values.tolist()], 
                      coords={'time': min_xr.time}, dims=('time',))
print(j_days)

<xarray.DataArray (time: 31)>
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31])
Coordinates:
  * time     (time) float64 3.78e+04 1.242e+05 2.106e+05 2.97e+05 3.834e+05 ...


In [235]:
#Create function to test whether the year of interest is a leap year
def leap_year(year):
    if calendar.isleap(int(year)) == True:
        print(str(year) +' is a leap year')
        ndy = 366
    else:
        print(str(year) +' is not a leap year')
        ndy = 365
    return(ndy)
#Use function and save the number of days within the year to a variable
ndy = leap_year( min_xr.time.units[14:18])
print(ndy)

1979 is not a leap year
365


# Calculate the difference in max and min temperature

In [236]:
#Create array of temperature difference
T_diff = max_xr.Tair - min_xr.Tair
print(T_diff)

<xarray.DataArray 'Tair' (time: 31, lat: 13, lon: 14)>
array([[[      nan,       nan, ...,       nan,       nan],
        [      nan,       nan, ...,       nan,       nan],
        ...,
        [17.170868, 15.867645, ...,  6.073151,  4.085083],
        [16.701416, 15.60083 , ...,  5.179626,  3.687134]],

       [[      nan,       nan, ...,       nan,       nan],
        [      nan,       nan, ...,       nan,       nan],
        ...,
        [19.31424 , 17.881805, ...,  6.780487,  4.735474],
        [18.743195, 16.871582, ...,  6.110413,  4.205872]],

       ...,

       [[      nan,       nan, ...,       nan,       nan],
        [      nan,       nan, ...,       nan,       nan],
        ...,
        [15.075378, 15.339081, ...,  5.641571,  6.302032],
        [14.562775, 14.036774, ...,  5.455078,  5.808136]],

       [[      nan,       nan, ...,       nan,       nan],
        [      nan,       nan, ...,       nan,       nan],
        ...,
        [26.65329 , 27.051208, ...,  6.023712,  

# Save latitude as radians

In [237]:
#Create array of latitude in radians
lat_rad = np.radians(min_xr.lat)
print(lat_rad)

<xarray.DataArray 'lat' (lat: 13)>
array([0.161443, 0.17017 , 0.178896, 0.187623, 0.19635 , 0.205076, 0.213803,
       0.222529, 0.231256, 0.239983, 0.248709, 0.257436, 0.266163],
      dtype=float32)
Coordinates:
  * lat      (lat) float32 9.25 9.75 10.25 10.75 11.25 11.75 12.25 12.75 ...


# Calculate variables associated with solar radiation at specific latitude

In [238]:
# Caclulate the inverse relative distance between the Earth and the Sun
# Unit: dr in radians
dr = 1 + 0.033 * np.cos(2 * np.pi * j_days / ndy)
print(dr.isel)

<bound method DataArray.isel of <xarray.DataArray (time: 31)>
array([1.032995, 1.03298 , 1.032956, 1.032922, 1.032878, 1.032824, 1.032761,
       1.032688, 1.032605, 1.032512, 1.03241 , 1.032298, 1.032177, 1.032046,
       1.031906, 1.031756, 1.031597, 1.031428, 1.031251, 1.031063, 1.030867,
       1.030662, 1.030447, 1.030224, 1.029991, 1.02975 , 1.029499, 1.02924 ,
       1.028973, 1.028696, 1.028412])
Coordinates:
  * time     (time) float64 3.78e+04 1.242e+05 2.106e+05 2.97e+05 3.834e+05 ...>


In [239]:
# Calculate solar declination
# Unit: sd in radians
sd = 0.409 * np.sin(2 * mt.pi * j_days / ndy - 1.39)
print(sd)

<xarray.DataArray (time: 31)>
array([-0.401008, -0.399564, -0.398001, -0.39632 , -0.394522, -0.392607,
       -0.390576, -0.388429, -0.386166, -0.38379 , -0.381299, -0.378696,
       -0.37598 , -0.373153, -0.370216, -0.367169, -0.364012, -0.360749,
       -0.357378, -0.353901, -0.350319, -0.346634, -0.342846, -0.338956,
       -0.334966, -0.330877, -0.326689, -0.322405, -0.318025, -0.313551,
       -0.308984])
Coordinates:
  * time     (time) float64 3.78e+04 1.242e+05 2.106e+05 2.97e+05 3.834e+05 ...


In [240]:
# Calculate a sunset angle input variable
# Unit: no unit
sa_X = 1 - (np.tan(lat_rad))**2 * ((np.tan(sd))**2)
print(sa_X)

<xarray.DataArray (lat: 13, time: 31)>
array([[0.995232, 0.99527 , 0.995311, ..., 0.997126, 0.997211, 0.997297],
       [0.994692, 0.994735, 0.994781, ..., 0.9968  , 0.996896, 0.996991],
       [0.994122, 0.994169, 0.99422 , ..., 0.996456, 0.996562, 0.996668],
       ...,
       [0.988406, 0.988498, 0.988598, ..., 0.99301 , 0.993219, 0.993428],
       [0.98754 , 0.98764 , 0.987747, ..., 0.992488, 0.992712, 0.992937],
       [0.986639, 0.986746, 0.986861, ..., 0.991945, 0.992185, 0.992426]])
Coordinates:
  * lat      (lat) float32 9.25 9.75 10.25 10.75 11.25 11.75 12.25 12.75 ...
  * time     (time) float64 3.78e+04 1.242e+05 2.106e+05 2.97e+05 3.834e+05 ...


In [241]:
# Calculate sunset angle
# Units: sa in radians
sa = mt.pi / 2 - np.arctan(-np.tan(lat_rad) * np.tan(sd) / np.sqrt(sa_X))
print(sa)

<xarray.DataArray (lat: 13, time: 31)>
array([[1.501691, 1.501969, 1.50227 , ..., 1.517157, 1.517965, 1.518787],
       [1.497878, 1.498172, 1.498489, ..., 1.514199, 1.515052, 1.515919],
       [1.494053, 1.494361, 1.494695, ..., 1.511232, 1.512129, 1.513042],
       ...,
       [1.462909, 1.463344, 1.463815, ..., 1.487093, 1.488355, 1.489639],
       [1.458938, 1.459389, 1.459877, ..., 1.484017, 1.485325, 1.486657],
       [1.454947, 1.455414, 1.455919, ..., 1.480925, 1.482281, 1.48366 ]])
Coordinates:
  * lat      (lat) float32 9.25 9.75 10.25 10.75 11.25 11.75 12.25 12.75 ...
  * time     (time) float64 3.78e+04 1.242e+05 2.106e+05 2.97e+05 3.834e+05 ...


In [242]:
# Calcualte extraterrestrial radiation
# Units: ra in MJ / (m^ day)
ra = 37.586 * dr * (sa * np.sin(lat_rad) * np.sin(sd) + np.cos(lat_rad) * np.cos(sd) * np.sin(sa))
print(ra)

<xarray.DataArray (time: 31, lat: 13)>
array([[31.538599, 31.291606, 31.042742, ..., 28.987134, 28.722438, 28.456096],
       [31.572136, 31.325744, 31.077474, ..., 29.02638 , 28.762217, 28.496402],
       [31.608088, 31.362349, 31.114724, ..., 29.06854 , 28.804958, 28.539716],
       ...,
       [33.256539, 33.045948, 32.833155, ..., 31.053829, 30.822072, 30.588303],
       [33.340672, 33.132117, 32.921346, ..., 31.1576  , 30.927714, 30.695798],
       [33.425747, 33.219278, 33.010577, ..., 31.262795, 31.034827, 30.804811]])
Coordinates:
  * time     (time) float64 3.78e+04 1.242e+05 2.106e+05 2.97e+05 3.834e+05 ...
  * lat      (lat) float32 9.25 9.75 10.25 10.75 11.25 11.75 12.25 12.75 ...


# Calculate Potential Evapo-Transpiration (PET)

In [249]:
# # Calculate Potential Evapotranspiration
# # Units: PET in mm/day

pet = 0.0009384 * (mean_xr.Tair + 17.8) * np.sqrt(T_diff) * ra
pet = xr.Dataset({'PET':pet})
pet.PET.attrs['units'] = 'mm/day'
print(pet)

<xarray.Dataset>
Dimensions:  (lat: 13, lon: 14, time: 31)
Coordinates:
  * time     (time) float64 3.78e+04 1.242e+05 2.106e+05 2.97e+05 3.834e+05 ...
  * lon      (lon) float32 101.75 102.25 102.75 103.25 103.75 104.25 104.75 ...
  * lat      (lat) float32 9.25 9.75 10.25 10.75 11.25 11.75 12.25 12.75 ...
Data variables:
    PET      (time, lat, lon) float64 nan nan nan nan nan nan 3.031 2.99 ...


In [250]:
pet.PET

<xarray.DataArray 'PET' (time: 31, lat: 13, lon: 14)>
array([[[     nan,      nan, ...,      nan,      nan],
        [     nan,      nan, ...,      nan,      nan],
        ...,
        [4.952039, 4.714145, ..., 2.545436, 2.066632],
        [4.871423, 4.67987 , ..., 2.269898, 1.999368]],

       [[     nan,      nan, ...,      nan,      nan],
        [     nan,      nan, ...,      nan,      nan],
        ...,
        [5.355132, 5.101752, ..., 2.703864, 2.229193],
        [5.2879  , 4.986188, ..., 2.48266 , 2.14067 ]],

       ...,

       [[     nan,      nan, ...,      nan,      nan],
        [     nan,      nan, ...,      nan,      nan],
        ...,
        [5.022806, 5.070326, ..., 2.700368, 2.906334],
        [4.957335, 4.890787, ..., 2.610068, 2.858268]],

       [[     nan,      nan, ...,      nan,      nan],
        [     nan,      nan, ...,      nan,      nan],
        ...,
        [6.533857, 6.406227, ..., 2.483016, 2.746853],
        [6.60129 , 6.148051, ..., 2.668827, 2.8538

In [225]:
#Save PET output as netcdf
pet.to_netcdf(path = str(input_folder) +'PET'+nc_name[12:])

In [217]:

# #Save time variable
# time_unit = datetime.strptime(min_xr.time.units[14:], '%Y-%m-%d %H:%M:%S') 
# time_seconds = int(min_xr.time.isel(time=time_index).values)
# date = time_unit + timedelta(seconds = time_seconds)
# print('date: '+str(date))

# #Save coordinates
# lat_deg = float(min_xr.isel(time = time_index).lat[lat_index].values)
# lon_deg = float(min_xr.isel(time = time_index).lon[lon_index].values)
# print('lat: '+str(lat_deg))
# print('lon: '+str(lon_deg))



# #Save temperature varaibles
# T_min =  float(min_xr.sel(time = time_seconds, lat = lat_deg, lon = lon_deg).Tair)
# T_mean = float(mean_xr.sel(time = time_seconds, lat = lat_deg, lon = lon_deg).Tair)
# T_max = float(max_xr.sel(time = time_seconds, lat = lat_deg, lon = lon_deg).Tair)
# print('Minimum temp: '+ str(T_min))
# print('Mean temp: '+ str(T_mean))
# print('Maximum temp: '+ str(T_max))

# # Calculate temperature difference
# # Units: T_diff in degrees C
# # Example: T_diff = 34.8 - 25.6 = 9.2 degrees C

# T_diff = T_max - T_min
# print('T_DIFF: ' +str(T_diff))

# # Convert latitude to radians
# # Unit: lat_rad in radians (northern hemisphere)
# # Example: lat_rad = (pi / 180) * 13.73 = + 0.240 radians

# lat_rad = (mt.pi / 180) * lat_deg 

# print('LAT_RAD'+str(lat_rad))

# # Convert date to julian day of the year
# # Unit: j_days in julian days
# # Example: j_days = days between ("15/4/2015","1/1/2015") + 1 = 105 julian days

# start_of_year = datetime.strptime('01/01/'+ str(date.year),'%m/%d/%Y') #save first day of year as variable
# j_days = int(((date - start_of_year)+timedelta(days=1))/timedelta(days=1)) #calculate julian days
# print('J_DAYS: '+str(j_days))

# Calculate the number of days in the year to account for leap year
# Units: ndy in days

# def leap_year(year):
#     if calendar.isleap(year) == True:
#         print(str(year) +' is a leap year')
#         ndy = 366
#     else:
#         print(str(year) +' is not a leap year')
#         ndy = 365
    

# print('NDY: '+str(ndy))

# Caclulate the inverse relative distance between the Earth and the Sun
# Unit: dr in radians
# Example: dr = 1 + 0.033* cos(2*pi*105/365) = 0.992 radians

# dr = 1 + 0.033 * mt.cos(2 * mt.pi * j_days / ndy)
# print('DR: '+str(dr))

# Calculate solar declination
# Unit: sd in radians
# Example: sd = 0.409 * sin(2 * pi * 105 / 365 - 1.39) = 0.166 radians

# sd = 0.409 * mt.sin(2 * mt.pi * j_days / 365 - 1.39)
# print('SD: '+str(sd))

# Calculate a sunset angle input variable
# Unit: no unit
# Example: sa_X = 1 - [tan(0.24)]^2 * [tan(0.166)]^2 = 0.998

# sa_X = 1 - (mt.tan(lat_rad))**2 * ((mt.tan(sd))**2)
# print('SA_X: '+str(sa_X))

# Calculate sunset angle
# Units: sa in radians
# Example: sa = pi / 2 - arctan(-tan(0.24) * tan(0.166) / sqrt(0.998)) = 1.612 radians

# sa = mt.pi / 2 - mt.atan(-mt.tan(lat_rad) * mt.tan(sd) / mt.sqrt(sa_X))
# print('SA: '+str(sa))

# Calcualte extraterrestrial radiation
# Units: ra in MJ / (m^ day)
# Example: ra = 37.586 * 0.992 * (1.612 * sin(0.24) * sin(0.166) + cos(0.24) * cos(0.166) * sin(1.612)) = 38.06 MJ / (m^ day)

# ra = 37.586 * dr * (sa * mt.sin(lat_rad) * mt.sin(sd) + mt.cos(lat_rad) * mt.cos(sd) * mt.sin(sa))
# print('RA: '+str(ra))


# Calculate Potential Evapotranspiration
# Units: PET in mm/day
# Example: PET = 0.0009384 * (30.2 + 17.8) * sqrt(9.2) * 38.06 = 5.2 mm/day
pet = 0.0009384 * (T_mean + 17.8) * mt.sqrt(T_diff) * ra
print('PET: '+str(pet))


####Create array with values
lat_deg
lon_deg
date
PET

test_array = np.nan()
#create an xarray
#create netcdf of PET


NameError: name 'T_mean' is not defined

In [218]:
# lat_shape = float(min_xr.lat.shape[0])
# lon_shape = float(min_xr.lon.shape[0])

In [ ]:
# for blah in min_xr.Tair:
#     print(print(blah.lat
    

In [ ]:
# min_xr.isel(time=0,lat = 0, lon = 0).Tair

In [ ]:
# result_array = np.empty((10, 10))

# num = 1

# for line in data_array:
#     result = do_stuff(line)
#     result_array = np.append(result_array, [result], axis=0)

In [ ]:
#  time.values[0]

In [ ]:
# [to_date(epoch, time_unit) for epoch in time.values.tolist()]

In [ ]:
# to_date()

In [ ]:
# time_unit = datetime.strptime(min_xr.time.units[14:], '%Y-%m-%d %H:%M:%S') 

In [ ]:
# #Save time variable
# time_unit = datetime.strptime(min_xr.time.units[14:], '%Y-%m-%d %H:%M:%S') 
# time_seconds = int(min_xr.time.isel(time=time_index).values)
# date = time_unit + timedelta(seconds = time_seconds)
# print('date: '+str(date))

# #Save coordinates
# lat_deg = float(min_xr.isel(time = time_index).lat[lat_index].values)
# lon_deg = float(min_xr.isel(time = time_index).lon[lon_index].values)
# print('lat: '+str(lat_deg))
# print('lon: '+str(lon_deg))

# #Save temperature varaibles
# T_min =  float(min_xr.sel(time = time_seconds, lat = lat_deg, lon = lon_deg).Tair)
# T_mean = float(mean_xr.sel(time = time_seconds, lat = lat_deg, lon = lon_deg).Tair)
# T_max = float(max_xr.sel(time = time_seconds, lat = lat_deg, lon = lon_deg).Tair)
# print('Minimum temp: '+ str(T_min))
# print('Mean temp: '+ str(T_mean))
# print('Maximum temp: '+ str(T_max))

## Calculate max and min temperature difference

In [ ]:
# # Calculate temperature difference
# # Units: T_diff in degrees C
# # Example: T_diff = 34.8 - 25.6 = 9.2 degrees C

# T_diff = T_max - T_min
# print(T_diff)

## Convert latitude

In [ ]:
# # Convert latitude to radians
# # Unit: lat_rad in radians (northern hemisphere)
# # Example: lat_rad = (pi / 180) * 13.73 = + 0.240 radians

# lat_rad = (mt.pi / 180) * lat_deg 

# print(lat_rad)

## Calculate days and dates

In [ ]:
# # Convert date to julian day of the year
# # Unit: j_days in julian days
# # Example: j_days = days between ("15/4/2015","1/1/2015") + 1 = 105 julian days

# start_of_year = datetime.strptime('01/01/'+ str(date.year),'%m/%d/%Y') #save first day of year as variable
# j_days = int(((date - start_of_year)+timedelta(days=1))/timedelta(days=1)) #calculate julian days
# print(j_days)

In [ ]:
# # Calculate the number of days in the year to account for leap year
# # Units: ndy in days

# if calendar.isleap(date.year) == True:
#     print(str(date.year) +' is a leap year')
#     ndy = 366
# else:
#     print(str(date.year) +' is not a leap year')
#     ndy = 365
    

# print('There are '+str(ndy)+' days')

## Calculate extraterrestrial radiation

In [ ]:
# # Caclulate the inverse relative distance between the Earth and the Sun
# # Unit: dr in radians
# # Example: dr = 1 + 0.033* cos(2*pi*105/365) = 0.992 radians

# dr = 1 + 0.033 * mt.cos(2 * mt.pi * j_days / ndy)
# print (dr)

In [ ]:
# # Calculate solar declination
# # Unit: sd in radians
# # Example: sd = 0.409 * sin(2 * pi * 105 / 365 - 1.39) = 0.166 radians

# sd = 0.409 * mt.sin(2 * mt.pi * 105 / 365 - 1.39)
# print(sd)

In [ ]:
# # Calculate a sunset angle input variable
# # Unit: no unit
# # Example: sa_X = 1 - [tan(0.24)]^2 * [tan(0.166)]^2 = 0.998

# sa_X = 1 - (mt.tan(lat_rad))**2 * ((mt.tan(sd))**2)
# print(sa_X)

In [ ]:
# # Calculate sunset angle
# # Units: sa in radians
# # Example: sa = pi / 2 - arctan(-tan(0.24) * tan(0.166) / sqrt(0.998)) = 1.612 radians

# sa = mt.pi / 2 - mt.atan(-mt.tan(lat_rad) * mt.tan(sd) / mt.sqrt(sa_X))
# print(sa)

In [ ]:
# # Calcualte extraterrestrial radiation
# # Units: ra in MJ / (m^ day)
# # Example: ra = 37.586 * 0.992 * (1.612 * sin(0.24) * sin(0.166) + cos(0.24) * cos(0.166) * sin(1.612)) = 38.06 MJ / (m^ day)

# ra = 37.586 * dr * (sa * mt.sin(lat_rad) * mt.sin(sd) + mt.cos(lat_rad) * mt.cos(sd) * mt.sin(sa))
# print(ra)

## Calculate Potential Evapotranspiration

In [ ]:
# # Calculate Potential Evapotranspiration
# # Units: PET in mm/day
# # Example: PET = 0.0009384 * (30.2 + 17.8) * sqrt(9.2) * 38.06 = 5.2 mm/day
# pet = 0.0009384 * (T_mean + 17.8) * mt.sqrt(T_diff) * ra
# print(pet)

In [ ]:
# #Input variables
# #latitude (in degrees and minutes)
# input_lat = '13 44 N'
# #date (in mm/dd/yyyy)
# input_date = '04/15/2015'
# #daily maximum temperature (degrees C)
# input_Tmax =  '34.8'
# #daily mean temperature (degrees C)
# input_Tmean = '30.2'
# #daily minimum temperature (degrees C)
# input_Tmin = '25.6'

In [ ]:
# #Reformat datatypes
# lat_deg = int(input_lat.split(' ')[0])
# lat_min = int(input_lat.split(' ')[1])
# T_max= float(input_Tmax)
# T_mean = float(input_Tmean)
# T_min = float(input_Tmin)
# date = datetime.strptime(input_date,'%m/%d/%Y') #convert to datetime format
# year = date.year #save year as variable

In [ ]:
#Convert latitude to decimal degree
#Unit: lat_deg in decimal degrees
#Example: lat_deg = 13 + 44 / 60 = 13.73 decimal degrees

# lat_deg = lat_deg + lat_min/int(60)

# print(lat_deg)